# BigQuery Public Data - Stack Overflow Dataset
Stack Overflow Dataset
- Updated on a quarterly basis, this BigQuery dataset includes an archive of Stack Overflow content, including posts, votes, tags, and badges. This dataset is updated to mirror the Stack Overflow content on the Internet Archive, and is also available through the Stack Exchange Data Explorer.

### Authenticating with Google BigQuery service account key file

Before connecting to the Jupyter server, do the following in terminal window:
```
export GOOGLE_APPLICATION_CREDENTIALS = "/your/file/path/[FILE_NAME].json"
```

```
pip install --upgrade google-cloud-bigquery

```



### Set up BigQuery client & load Google Cloud BigQuery extension

In [1]:
from google.cloud import bigquery
client = bigquery.Client()
%load_ext google.cloud.bigquery

In [2]:
import statistics
import numpy as np
import pandas as pd

### Preliminary Stuff

#### What tables do the stack overflow dataset have?

In [3]:
data_ref = client.dataset("stackoverflow", project="bigquery-public-data")
data = client.get_dataset(data_ref)

tables = list(client.list_tables(data))
for table in tables:
    print(table.table_id)

badges
comments
post_history
post_links
posts_answers
posts_moderator_nomination
posts_orphaned_tag_wiki
posts_privilege_wiki
posts_questions
posts_tag_wiki
posts_tag_wiki_excerpt
posts_wiki_placeholder
stackoverflow_posts
tags
users
votes


#### Let's take a sneak peak of the posts_questions table

In [4]:
table_ref = data_ref.table("posts_questions")
table = client.get_table(table_ref)
client.list_rows(table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,56412356,S3 Multipart upload with pause and resume func...,<p>I am trying to acheive s3 multipart upload ...,None,0,0,None,2019-06-02 05:09:24.723000+00:00,None,2019-06-02 05:09:24.723000+00:00,None,None,None,None,7225816,None,1,0,<javascript><amazon-s3><stream>,1
1,56318079,Angular Nginx Docker 404,<p>Been driving myself nuts trying to figure t...,None,1,0,None,2019-05-26 23:14:43.403000+00:00,None,2019-05-26 23:21:11.503000+00:00,None,None,None,None,1058951,None,1,0,<angular><docker><nginx><http-status-code-404>,257
2,56136152,Switch structure control for OSC msg,<p>I’m new to SC and the whole music programmi...,None,0,0,None,2019-05-14 18:05:32.077000+00:00,None,2019-05-14 18:05:32.077000+00:00,None,None,None,None,8880735,None,1,0,<supercollider>,2
3,56207713,AWS QuickSight - Smooth line graph/plot,<p>do you know if ''''AWS QuickSight'''' has a...,None,0,0,None,2019-05-19 12:15:57.697000+00:00,None,2019-05-19 12:15:57.697000+00:00,None,None,None,None,11036699,None,1,0,<amazon>,2
4,56250259,I am not able to access kubernetes dash-board ...,<p>I have installed docker and kubernetes in m...,None,0,0,None,2019-05-22 06:03:36.240000+00:00,None,2019-05-22 06:03:36.240000+00:00,None,None,None,None,9493974,None,1,0,<kubernetes-dashboard>,2


#### Let's take a sneak peak of the posts_answers table

In [5]:
table_ref = data_ref.table("posts_answers")
table = client.get_table(table_ref)
client.list_rows(table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,55832858,None,<p>There may be privacy description <em>(for S...,None,None,0,None,2019-04-24 14:45:59.090000+00:00,None,2019-04-24 14:45:59.090000+00:00,None,None,None,None,7266317,6004032,2,0,None,None
1,55832872,None,<p>How do I set the name of the button to the ...,None,None,3,None,2019-04-24 14:46:52.690000+00:00,None,2019-04-24 14:46:52.690000+00:00,None,None,None,None,5873109,55832746,2,0,None,None
2,55832876,None,<pre><code>go test -v -timeout 30s &lt;path_to...,None,None,0,None,2019-04-24 14:47:09.227000+00:00,None,2019-04-24 14:47:09.227000+00:00,None,None,None,None,5713047,16935965,2,0,None,None
3,55832878,None,<p>I suggest you create one custom pin represe...,None,None,0,None,2019-04-24 14:47:10.297000+00:00,None,2019-04-24 14:47:10.297000+00:00,None,None,None,None,6001090,33622927,2,0,None,None
4,55832879,None,"<p>You have to use <a href=""https://en.cpprefe...",None,None,2,None,2019-04-24 14:47:16.853000+00:00,None,2019-04-24 14:47:16.853000+00:00,None,None,None,None,10765031,55831782,2,0,None,None


### BigQuery SQL

#### Review table schema

In [6]:
# comments table schema
client.get_table(data_ref.table("posts_questions")).schema

[SchemaField('id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('title', 'STRING', 'NULLABLE', None, ()),
 SchemaField('body', 'STRING', 'NULLABLE', None, ()),
 SchemaField('accepted_answer_id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('answer_count', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('comment_count', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('community_owned_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('creation_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('favorite_count', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('last_activity_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('last_edit_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('last_editor_display_name', 'STRING', 'NULLABLE', None, ()),
 SchemaField('last_editor_user_id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('owner_display_name', 'STRING', 'NULLABLE', None, ()),
 SchemaField('owner_user_id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('parent_id'

In [7]:
# stories table schema
client.get_table(data_ref.table("posts_answers")).schema

[SchemaField('id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('title', 'STRING', 'NULLABLE', None, ()),
 SchemaField('body', 'STRING', 'NULLABLE', None, ()),
 SchemaField('accepted_answer_id', 'STRING', 'NULLABLE', None, ()),
 SchemaField('answer_count', 'STRING', 'NULLABLE', None, ()),
 SchemaField('comment_count', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('community_owned_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('creation_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('favorite_count', 'STRING', 'NULLABLE', None, ()),
 SchemaField('last_activity_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('last_edit_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('last_editor_display_name', 'STRING', 'NULLABLE', None, ()),
 SchemaField('last_editor_user_id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('owner_display_name', 'STRING', 'NULLABLE', None, ()),
 SchemaField('owner_user_id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('parent_id', '

#### How long does it generally take (in seconds) for questions to receive answers? Let's look at January 2018 only for this demo.

Approach 1 - using CTE

In [8]:
query = """
        WITH first_response AS
        (
        SELECT 
            parent_id, 
            MIN(creation_date) AS first_response_date
        FROM 
            `bigquery-public-data.stackoverflow.posts_answers` 
        GROUP BY
            parent_id
        HAVING
            MIN(creation_date) >= '2018-01-01'
        )
        SELECT 
            q.id, 
            TIMESTAMP_DIFF(a.first_response_date, q.creation_date, MINUTE) AS response_time
        FROM 
            `bigquery-public-data.stackoverflow.posts_questions` q
            LEFT JOIN first_response a
            ON q.id = a.parent_id
        WHERE
            creation_date >= '2018-01-01'
            AND 
            creation_date < '2018-02-01'
        ORDER BY
            response_time DESC
        """

result = client.query(query).result().to_dataframe()

In [9]:
print("Number of questions: {:,.0f}".format(len(result)))
print("Percentage of answered questions: {:.0%}".format(
    sum(result["response_time"].notnull()) / len(result)))
print("-" * 30)
print(result.response_time.describe())

Number of questions: 163,342
Percentage of answered questions: 82%
------------------------------
count    134011.000000
mean      11769.882928
std       61523.008731
min           0.000000
25%          10.000000
50%          43.000000
75%         429.000000
max      735651.000000
Name: response_time, dtype: float64


Approach 2

In [10]:
query = """
        SELECT
            q.id,
            MIN(TIMESTAMP_DIFF(a.creation_date, q.creation_date, MINUTE)) AS response_time
        FROM
            `bigquery-public-data.stackoverflow.posts_questions` q
        LEFT JOIN
            `bigquery-public-data.stackoverflow.posts_answers` a
            ON a.parent_id = q.id
        WHERE 
            q.creation_date >= '2018-01-01' and q.creation_date < '2018-02-01'
        GROUP BY
            q.id
        ORDER BY 
            response_time
        """

result = client.query(query).result().to_dataframe()

In [11]:
print("Number of questions: {:,.0f}".format(len(result)))
print("Percentage of answered questions: {:.0%}".format(
    sum(result["response_time"].notnull()) / len(result)))
print("-" * 30)
print(result.response_time.describe())

Number of questions: 163,342
Percentage of answered questions: 82%
------------------------------
count    134011.000000
mean      11769.882928
std       61523.008731
min           0.000000
25%          10.000000
50%          43.000000
75%         429.000000
max      735651.000000
Name: response_time, dtype: float64


#### New user experiences 
Let's try to understand better the initial experiences that **new users** typically have with the Stack Overflow website. 
- Is it more common for them to ask or answer questions? 
- After signing up, how long does it take for **new users** to first interact with the website (if ever)?
- For this, we need to use a third table: the ```users``` table
- To answer the questions, we need a table showing:
    - ```id```: the IDs of all users who created their account in January 2019
    - ```account_creation_date```: The time stamp when the user first created the account. 
    - ```first_question```: the first time the user posted a question on the website. If the user had never posted a question, it should show NULL.
    - ```first_answer```: the first time the user answered a question on the website. If the user had never posted a question, it should show NULL.

Review ```users``` table schema
- The relevant columns from the ```users``` table are ```id``` (the ID of each user) and ```creation_date``` (when the user joined the Stack Overflow site, in DATETIME format).

In [12]:
client.get_table(data_ref.table("users")).schema

[SchemaField('id', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('display_name', 'STRING', 'NULLABLE', None, ()),
 SchemaField('about_me', 'STRING', 'NULLABLE', None, ()),
 SchemaField('age', 'STRING', 'NULLABLE', None, ()),
 SchemaField('creation_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('last_access_date', 'TIMESTAMP', 'NULLABLE', None, ()),
 SchemaField('location', 'STRING', 'NULLABLE', None, ()),
 SchemaField('reputation', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('up_votes', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('down_votes', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('views', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('profile_image_url', 'STRING', 'NULLABLE', None, ()),
 SchemaField('website_url', 'STRING', 'NULLABLE', None, ())]

Approach 1 - CTE

In [13]:
query = """
        WITH new_user AS 
        (
        SELECT
            id,
            creation_date AS account_creation_date
        FROM
            `bigquery-public-data.stackoverflow.users` u
        WHERE 
            creation_date >= '2019-01-01' 
            AND
            creation_date < '2019-02-01'
        )
        SELECT
            n.id,
            n.account_creation_date AS account_creation_date,
            MIN(q.creation_date) AS first_question,
            MIN(a.creation_date) AS first_answer
        FROM
            new_user n 
            LEFT JOIN `bigquery-public-data.stackoverflow.posts_questions` q
            ON n.id = q.owner_user_id
            LEFT JOIN `bigquery-public-data.stackoverflow.posts_answers` a
            ON n.id = a.owner_user_id
        GROUP BY
            1, 2
        ORDER BY 
            1
        """

result = client.query(query).result().to_dataframe()

In [14]:
result.tail(10)

,id,account_creation_date,first_question,first_answer
142889,10998583,2019-01-31 23:58:54.817000+00:00,NaT,NaT
142890,10998584,2019-01-31 23:59:03.937000+00:00,NaT,NaT
142891,10998585,2019-01-31 23:59:12.210000+00:00,NaT,NaT
142892,10998586,2019-01-31 23:59:18.160000+00:00,NaT,NaT
142893,10998587,2019-01-31 23:59:29.003000+00:00,2019-04-15 03:12:33.887000+00:00,NaT
142894,11040013,2019-01-21 05:30:56.187000+00:00,NaT,NaT
142895,11226695,2019-01-11 10:53:19.810000+00:00,NaT,NaT
142896,11231243,2019-01-11 05:10:36.537000+00:00,2019-01-28 07:45:01.560000+00:00,2019-01-31 05:56:53.747000+00:00
142897,11486952,2019-01-27 16:49:14.507000+00:00,NaT,2019-05-14 05:41:36.487000+00:00
142898,11525772,2019-01-18 22:06:20.743000+00:00,2019-05-20 02:00:18.237000+00:00,2019-01-18 22:06:20.743000+00:00


Approach 2

In [15]:
query = """
        SELECT
            n.id AS id,
            n.creation_date AS account_creation_date,
            MIN(q.creation_date) AS first_question,
            MIN(a.creation_date) AS first_answer
        FROM
            `bigquery-public-data.stackoverflow.users` n
            LEFT JOIN `bigquery-public-data.stackoverflow.posts_questions` q
            ON n.id = q.owner_user_id
            LEFT JOIN `bigquery-public-data.stackoverflow.posts_answers` a 
            ON n.id = a.owner_user_id    
        WHERE
            n.creation_date >= '2019-01-01'
            AND n.creation_date < '2019-02-01'
        GROUP BY
            1, 2
        ORDER BY 
            1
        """

result = client.query(query).result().to_dataframe()

In [16]:
result.tail(10)

,id,account_creation_date,first_question,first_answer
142889,10998583,2019-01-31 23:58:54.817000+00:00,NaT,NaT
142890,10998584,2019-01-31 23:59:03.937000+00:00,NaT,NaT
142891,10998585,2019-01-31 23:59:12.210000+00:00,NaT,NaT
142892,10998586,2019-01-31 23:59:18.160000+00:00,NaT,NaT
142893,10998587,2019-01-31 23:59:29.003000+00:00,2019-04-15 03:12:33.887000+00:00,NaT
142894,11040013,2019-01-21 05:30:56.187000+00:00,NaT,NaT
142895,11226695,2019-01-11 10:53:19.810000+00:00,NaT,NaT
142896,11231243,2019-01-11 05:10:36.537000+00:00,2019-01-28 07:45:01.560000+00:00,2019-01-31 05:56:53.747000+00:00
142897,11486952,2019-01-27 16:49:14.507000+00:00,NaT,2019-05-14 05:41:36.487000+00:00
142898,11525772,2019-01-18 22:06:20.743000+00:00,2019-05-20 02:00:18.237000+00:00,2019-01-18 22:06:20.743000+00:00


Data Frame operations

In [17]:
def first_interaction(df):
    if pd.isnull(df.first_question) & pd.isnull(df.first_answer):
        return np.nan
    elif pd.isnull(df.first_question) & pd.isnull(df.first_answer) == False:
        return df.first_answer
    elif pd.isnull(df.first_question) == False & pd.isnull(df.first_answer):
        return df.first_question
    else:
        return min(df.first_question, df.first_answer)

result['first_interaction'] = result.apply(first_interaction, axis = 1)
result['interval_mins'] = (result['first_interaction'] - result['account_creation_date']).dt.total_seconds() // 60
result['interval_hours'] = result['interval_mins'] // 60
# drop two problematic rows where interaction timestamp is before account creation
result.drop(result.index[[81066, 142872]], inplace=True) 

In [18]:
result.tail()

,id,account_creation_date,first_question,first_answer,first_interaction,interval_mins,interval_hours
142894,11040013,2019-01-21 05:30:56.187000+00:00,NaT,NaT,NaT,NaN,NaN
142895,11226695,2019-01-11 10:53:19.810000+00:00,NaT,NaT,NaT,NaN,NaN
142896,11231243,2019-01-11 05:10:36.537000+00:00,2019-01-28 07:45:01.560000+00:00,2019-01-31 05:56:53.747000+00:00,2019-01-31 05:56:53.747000+00:00,28846.0,480.0
142897,11486952,2019-01-27 16:49:14.507000+00:00,NaT,2019-05-14 05:41:36.487000+00:00,2019-05-14 05:41:36.487000+00:00,153412.0,2556.0
142898,11525772,2019-01-18 22:06:20.743000+00:00,2019-05-20 02:00:18.237000+00:00,2019-01-18 22:06:20.743000+00:00,2019-01-18 22:06:20.743000+00:00,0.0,0.0


Answers

In [19]:
print(
    """
    {A:,.0f} users created their account on Stack Overflow in January 2019. \n\
    Among the {A:,.0f} new users, {B:,.0f} asked questions without answering any; \n\
    {C:,.0f} answered questions without asking any;
    and {D:,.0f} users did both. 
    """.format(
        A = result.account_creation_date.count(),
        B = result[result.first_question.notnull() & result.first_answer.isnull()]['id'].count(),
        C = result[result.first_question.isnull() & result.first_answer.notnull()]['id'].count(),
        D = result[result['first_question'].notnull() & result['first_answer'].notnull()]['id'].count()
        )
)
print("-" * 30)
print(result.interval_hours.describe())
print("-" * 30)
print("Half of the all questions were answered in less than {:.0f} hours or less.".format(result.interval_hours.median()))


    142,897 users created their account on Stack Overflow in January 2019. 
    Among the 142,897 new users, 23,064 asked questions without answering any; 
    5,642 answered questions without asking any;
    and 4,319 users did both. 
    
------------------------------
count    9961.000000
mean      422.059833
std       773.492667
min         0.000000
25%         0.000000
50%        12.000000
75%       455.000000
max      3552.000000
Name: interval_hours, dtype: float64
------------------------------
Half of the all questions were answered in less than 12 hours or less.


#### Lastly, how many distinct users posted, either a question or a answer, on New Year's Day of 2019?

In [20]:
query = """
        SELECT
            owner_user_id AS user_id
        FROM
            `bigquery-public-data.stackoverflow.posts_questions` 
        WHERE
            EXTRACT(DATE FROM creation_date) = '2019-01-01'
        UNION DISTINCT
        SELECT
            owner_user_id AS user_id
        FROM
            `bigquery-public-data.stackoverflow.posts_answers` 
        WHERE
            EXTRACT(DATE FROM creation_date) = '2019-01-01'
        """

result = client.query(query).result().to_dataframe()
result.head()

,user_id
0,7347610.0
1,2074297.0
2,10658268.0
3,427157.0
4,5786458.0


In [21]:
print("{:,.0f} distinct users posted on Stack Overflow on New Year's Day of 2019".format(result.user_id.count()))

4,754 distinct users posted on Stack Overflow on New Year's Day of 2019


# End of Session